<a href="https://colab.research.google.com/github/amit306/machineLearning/blob/main/GermanDataSetFairNessCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
#This notebook will use German database and perform the following steps.
#1. Load & preprocess German data
#2. Define metrics
#3. Train 4 individual models
 #   a. LR + Reweighing
 #   b. RF + Equalized Odds
 #   c. Adversarial Debiasing (DNN)
 #   d. Deep NN with Embedding
#4. Evaluate each model: accuracy + fairness
#5. Create fairness-weighted ensemble
#6. Evaluate ensemble
#7. Run Counterfactual Fairness audit on ensemble


In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [32]:
#load data from http URL
column_names = [
    "status", "duration", "credit_history", "purpose", "credit_amount",
    "savings", "employment", "installment_rate", "personal_status_sex",
    "other_debtors", "residence_since", "property", "age", "other_installment_plans",
    "housing", "number_credits", "job", "people_liable", "telephone",
    "foreign_worker", "class"]

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"

data = pd.read_csv(url, sep=' ', names=column_names)
data.head()

,status,duration,credit_history,purpose,credit_amount,savings,employment,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,class
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [33]:
# Step 1: Convert target class: 1 = Good credit, 2 = Bad credit → 1 and 0
data['class'] = data['class'].map({1: 1, 2: 0})

# Step 2: Create binary sensitive attributes
# 2.1 Age Binary: 1 if age >= 25 else 0
data['age_binary'] = data['age'].apply(lambda x: 1 if x >= 25 else 0)

# 2.2 Gender from personal_status_sex
# Mapping based on UCI docs: A91, A93, A94 → male; A92, A95 → female
data['gender'] = data['personal_status_sex'].map(
    {'A91': 'male', 'A93': 'male', 'A94': 'male', 'A92': 'female', 'A95': 'female'}
)

# 2.3 Binary encode gender
data['gender_binary'] = data['gender'].map({'male': 1, 'female': 0})

# 2.4 Convert foreign_worker to binary
data['foreign_worker_binary'] = data['foreign_worker'].map({'A201': 1, 'A202': 0})

# Preview again
data[['age', 'age_binary', 'gender', 'gender_binary', 'foreign_worker', 'foreign_worker_binary']].head()


,age,age_binary,gender,gender_binary,foreign_worker,foreign_worker_binary
0,67,1,male,1,A201,1
1,22,0,female,0,A201,1
2,49,1,male,1,A201,1
3,45,1,male,1,A201,1
4,53,1,male,1,A201,1


In [34]:
# Drop unused categorical columns for now
X = data.drop(columns=['class', 'personal_status_sex', 'foreign_worker', 'gender', 'foreign_worker_binary'])

# One-hot encode all categorical features
X = pd.get_dummies(X, drop_first=True)

# Target and sensitive
y = data['class']
sensitive_attr = data['age_binary']

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(
    X, y, sensitive_attr, test_size=0.3, random_state=42, stratify=y
)

In [36]:
!pip install -q aif360

from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Helper to convert to aif360 dataset
def to_aif360(X, y, s):
    df = X.copy()
    df['label'] = y.values
    df['sensitive'] = s.values
    return BinaryLabelDataset(df=df, label_names=['label'], protected_attribute_names=['sensitive'])

# AIF360 conversion
aif_train = to_aif360(X_train, y_train, s_train)
aif_test = to_aif360(X_test, y_test, s_test)

# Reweighing
RW = Reweighing(unprivileged_groups=[{'sensitive': 0}], privileged_groups=[{'sensitive': 1}])
RW.fit(aif_train)
aif_train_rw = RW.transform(aif_train)

# Logistic Regression with instance weights
lr = LogisticRegression(max_iter=500)
lr.fit(X_train, y_train, sample_weight=aif_train_rw.instance_weights)

# Predict
y_pred_lr = lr.predict(X_test)
acc = accuracy_score(y_test, y_pred_lr)
print(f"✅ Accuracy (Reweighing + LR): {acc:.4f}")


✅ Accuracy (Reweighing + LR): 0.7400


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
aif_pred = to_aif360(X_test, pd.Series(y_pred_lr), s_test.reset_index(drop=True))
metric = ClassificationMetric(aif_test, aif_pred,
                              unprivileged_groups=[{'sensitive': 0}],
                              privileged_groups=[{'sensitive': 1}])

print("📊 Fairness Metrics (Reweighing + Logistic Regression):")
print(f"Demographic Parity Difference : {metric.statistical_parity_difference():.4f}")
print(f"Equal Opportunity Difference  : {metric.equal_opportunity_difference():.4f}")


📊 Fairness Metrics (Reweighing + Logistic Regression):
Demographic Parity Difference : 0.0071
Equal Opportunity Difference  : -0.0125


In [38]:
# ✅ 1. Install if needed
!pip install -q fairlearn

# ✅ 2. Import required packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import (
    demographic_parity_difference,
    equalized_odds_difference,
)

# ✅ 3. Fit the Random Forest on training data
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# ✅ 4. Fit ThresholdOptimizer on training set
postproc = ThresholdOptimizer(
    estimator=rf,
    constraints="equalized_odds",
    prefit=True
)

# Use age_binary as sensitive feature
postproc.fit(X_train, y_train, sensitive_features=s_train)

# ✅ 5. Predict on test set
y_pred_rf = postproc.predict(X_test, sensitive_features=s_test)

# ✅ 6. Accuracy
accuracy = accuracy_score(y_test, y_pred_rf)
print(f"✅ Accuracy (Random Forest + Equalized Odds): {accuracy:.4f}")

# ✅ 7. Fairness Metrics
dpd = demographic_parity_difference(y_test, y_pred_rf, sensitive_features=s_test)
eod = equalized_odds_difference(y_test, y_pred_rf, sensitive_features=s_test)

print("📊 Fairness Metrics (Random Forest + Equalized Odds):")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equalized Odds Difference   : {eod:.4f}")

# ✅ 8. Debug: Prediction Distribution
print("Prediction distribution:", pd.Series(y_pred_rf).value_counts())
print("Sensitive group sizes:\n", s_test.value_counts())


✅ Accuracy (Random Forest + Equalized Odds): 0.7267
📊 Fairness Metrics (Random Forest + Equalized Odds):
Demographic Parity Difference: 0.1917
Equalized Odds Difference   : 0.2099
Prediction distribution: 1    240
0     60
Name: count, dtype: int64
Sensitive group sizes:
 age_binary
1    253
0     47
Name: count, dtype: int64


In [39]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Identify categorical and numeric columns
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()
numeric_cols = X_train.select_dtypes(include='number').columns.tolist()

# Encode categorical variables
encoders = {}
for col in categorical_cols:
    enc = LabelEncoder()
    X_train[col] = enc.fit_transform(X_train[col])
    X_test[col] = enc.transform(X_test[col])
    encoders[col] = enc

# Normalize numeric columns
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])


In [40]:
# Convert to tensors
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test.values, dtype=tf.float32)


In [41]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Class weights to handle imbalance (optional)
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
cw = {0: class_weights[0], 1: class_weights[1]}

# Train
model.fit(X_train_tensor, y_train_tensor, epochs=15, batch_size=32, class_weight=cw, verbose=1)


Epoch 1/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.5897 - loss: 0.7159
Epoch 2/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.6027 - loss: 0.6618
Epoch 3/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6823 - loss: 0.6253
Epoch 4/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6483 - loss: 0.6439
Epoch 5/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6888 - loss: 0.6228
Epoch 6/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7131 - loss: 0.6113
Epoch 7/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7214 - loss: 0.5786
Epoch 8/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7442 - loss: 0.5461
Epoch 9/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7625 - loss: 0.5213
Epoch 10/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7627 - loss: 0.5119
Epoch 11/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7444 - loss: 0.5214
Epoch 12/15
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy:

In [42]:
from sklearn.metrics import accuracy_score
from fairlearn.metrics import (
    demographic_parity_difference, equalized_odds_difference
)

y_pred_probs = model.predict(X_test_tensor).flatten()
y_pred_dnn = (y_pred_probs >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred_dnn)
dpd = demographic_parity_difference(y_test, y_pred_dnn, sensitive_features=s_test)
eod = equalized_odds_difference(y_test, y_pred_dnn, sensitive_features=s_test)

print(f"\n✅ Accuracy (DNN): {acc:.4f}")
print("📊 Fairness Metrics (DNN):")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equalized Odds Difference   : {eod:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

✅ Accuracy (DNN): 0.7200
📊 Fairness Metrics (DNN):
Demographic Parity Difference: 0.3394
Equalized Odds Difference   : 0.3265


In [43]:
#Deep NN with Embedding Inputs (TensorFlow)

In [44]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Drop target from features
features = data.drop(columns=['class'])
labels = (data['class'] == 1).astype(int)  # 1 = good credit

# Define sensitive feature
data['age_binary'] = (data['age'] >= 25).astype(int)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.3, random_state=42, stratify=labels
)

s_train = X_train['age_binary']
s_test = X_test['age_binary']

# Define categorical and numerical columns
cat_cols = X_train.select_dtypes(include='object').columns.tolist()
num_cols = X_train.select_dtypes(include='number').drop('age_binary', axis=1).columns.tolist()


In [45]:
inputs = []
encoded_features = []

# Embedding for categorical features
for col in cat_cols:
    vocab = X_train[col].unique()
    vocab_size = len(vocab) + 1

    input_col = tf.keras.Input(shape=(1,), name=col, dtype=tf.string)
    embedding = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='int')(input_col)
    embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=4)(embedding)
    embedding = tf.keras.layers.Reshape((4,))(embedding)

    inputs.append(input_col)
    encoded_features.append(embedding)

# Normalized numerical features
for col in num_cols:
    input_col = tf.keras.Input(shape=(1,), name=col)
    norm = tf.keras.layers.Normalization(axis=None)
    norm.adapt(X_train[col].values.reshape(-1, 1))

    inputs.append(input_col)
    encoded_features.append(norm(input_col))

# Combine all features
all_features = tf.keras.layers.concatenate(encoded_features)

# Final DNN
x = tf.keras.layers.Dense(64, activation='relu')(all_features)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [46]:
# Convert DataFrame to dictionary of inputs for Keras
def df_to_dict(df):
    return {col: df[col].values for col in df.columns}

# Combine categorical and numerical column lists
selected_cols = cat_cols + num_cols

# Use the combined list to select columns from X_train and X_test
train_dict = df_to_dict(X_train[selected_cols])
test_dict = df_to_dict(X_test[selected_cols])

# Compute class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
cw = {0: class_weights[0], 1: class_weights[1]}

In [47]:
model.fit(train_dict, y_train, epochs=20, batch_size=32, class_weight=cw, verbose=1)

# Predict
y_pred_probs = model.predict(test_dict).flatten()
y_pred_dnn_embed = (y_pred_probs >= 0.5).astype(int)


Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.6025 - loss: 0.6697
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6191 - loss: 0.6573
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6274 - loss: 0.6327
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6462 - loss: 0.6276
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6616 - loss: 0.6206
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7212 - loss: 0.5983
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7269 - loss: 0.5628
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7145 - loss: 0.5569
Epoch 9/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7399 - loss: 0.5233
Epoch 10/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7259 - loss: 0.5315
Epoch 11/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7364 - loss: 0.5077
Epoch 12/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7539 - l

In [48]:
from sklearn.metrics import accuracy_score
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference

acc = accuracy_score(y_test, y_pred_dnn_embed)
dpd = demographic_parity_difference(y_test, y_pred_dnn_embed, sensitive_features=s_test)
eod = equalized_odds_difference(y_test, y_pred_dnn_embed, sensitive_features=s_test)

print(f"✅ Accuracy (DNN with Embeddings): {acc:.4f}")
print("📊 Fairness Metrics:")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equalized Odds Difference   : {eod:.4f}")


✅ Accuracy (DNN with Embeddings): 0.7167
📊 Fairness Metrics:
Demographic Parity Difference: 0.2361
Equalized Odds Difference   : 0.1731


In [49]:
#Create fairness-weighted ensemble

In [50]:
# Model predictions (already computed in your previous steps)
pred_lr = y_pred_lr
pred_rf = y_pred_rf
pred_dnn = y_pred_dnn
pred_dnn_embed = y_pred_dnn_embed

# DPDs from previous results
dpd_lr = 0.0071
dpd_rf = 0.0001  # use small epsilon to avoid zero division
dpd_dnn = 0.2929
dpd_dnn_embed = 0.1534

# Add a small epsilon
epsilon = 1e-6

# Compute weights (inverse of DPD)
raw_weights = np.array([
    1 / (dpd_lr + epsilon),
    1 / (dpd_rf + epsilon),
    1 / (dpd_dnn + epsilon),
    1 / (dpd_dnn_embed + epsilon)
])

# Normalize weights to sum to 1
weights = raw_weights / raw_weights.sum()
weights_dict = {
    "LR": weights[0],
    "RF": weights[1],
    "DNN": weights[2],
    "DNN+Embed": weights[3]
}
print("🎯 Model Weights Based on Fairness:\n", weights_dict)


🎯 Model Weights Based on Fairness:
 {'LR': np.float64(0.014010024067249632), 'RF': np.float64(0.985001791104353), 'DNN': np.float64(0.0003396546304093863), 'DNN+Embed': np.float64(0.0006485301979878857)}


In [51]:
# Stack predictions into array
pred_matrix = np.stack([pred_lr, pred_rf, pred_dnn, pred_dnn_embed], axis=1)

# Weighted vote: multiply predictions by model weights
weighted_votes = np.dot(pred_matrix, weights)

# Final prediction: if weighted vote ≥ 0.5 → 1, else 0
y_pred_ensemble = (weighted_votes >= 0.5).astype(int)


In [52]:
from sklearn.metrics import accuracy_score
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference

# Final metrics
acc = accuracy_score(y_test, y_pred_ensemble)
dpd = demographic_parity_difference(y_test, y_pred_ensemble, sensitive_features=s_test)
eod = equalized_odds_difference(y_test, y_pred_ensemble, sensitive_features=s_test)

print("\n✅ Fairness-Weighted Ensemble Results:")
print(f"Accuracy: {acc:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equalized Odds Difference: {eod:.4f}")



✅ Fairness-Weighted Ensemble Results:
Accuracy: 0.7267
Demographic Parity Difference: 0.1917
Equalized Odds Difference: 0.2099


In [ ]:
#Run Counterfactual Fairness audit on ensemble

In [2]:
!pip install dice-ml


In [11]:
!pip install dice-ml
